In [ ]:
from google.cloud import bigquery
from google.cloud import aiplatform
import bigframes.pandas as bpd
import pandas as pd
from vertexai.language_models._language_models import TextGenerationModel
from vertexai.generative_models import GenerativeModel
from bigframes.ml.cluster import KMeans
from bigframes.ml.model_selection import train_test_split

In [ ]:
project_id = 'qwiklabs-gcp-03-30f9ddcee663'
dataset_name = "ecommerce"
model_name = "customer_segmentation_model"
table_name = "customer_stats"
location = "us-central1"
client = bigquery.Client(project=project_id)
aiplatform.init(project=project_id, location=location)

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE ecommerce.customer_stats AS
SELECT
  user_id,
  DATE_DIFF(CURRENT_DATE(), CAST(MAX(order_created_date) AS DATE), day) AS days_since_last_order, ---RECENCY
  COUNT(order_id) AS count_orders, --FREQUENCY
  AVG(sale_price) AS average_spend --MONETARY
  FROM (
      SELECT
        user_id,
        order_id,
        sale_price,
        created_at AS order_created_date
        FROM `bigquery-public-data.thelook_ecommerce.order_items`
        WHERE
        created_at
            BETWEEN '2022-01-01' AND '2023-01-01'
  )
GROUP BY user_id;

In [ ]:
# prompt: Convert the table ecommerce.customer_stats to a bigframes dataframe and show the top 10 records

# Read the BigQuery table "ecommerce.customer_stats" into a BigFrames DataFrame.
df = bpd.read_gbq("qwiklabs-gcp-03-30f9ddcee663.ecommerce.customer_stats")

# Display the first 10 rows of the DataFrame.
result = df.head(10)

In [ ]:
# prompt: 1. Split the DataFrame df into training and testing sets using train_test_split with a test size of 0.2 and random_state=42 for reproducibility. Store the splits as df_train and df_test. 2. Create a KMeans clustering model using bigframes.ml.cluster.KMeans with 5 clusters (note: do not include random_state since it's unsupported). 3. Fit the KMeans model on the training data (df_train). 4. Save the trained model to BigQuery using the to_gbq method, specifying the model path as project_id.dataset_name.model_name. 5. Store the trained model object in a variable named result.

# Split the DataFrame df into training and testing sets.
# We use train_test_split from bigframes.ml.model_selection, specifying the test_size and random_state for reproducibility.
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Create a KMeans clustering model.
# We instantiate KMeans from bigframes.ml.cluster, setting the number of clusters to 5.
kmeans_model = KMeans(n_clusters=5)

# Fit the KMeans model on the training data.
# The fit method trains the model using the features in df_train.
kmeans_model.fit(df_train)

# Save the trained model to BigQuery.
# The to_gbq method saves the model to the specified BigQuery path.
kmeans_model.to_gbq(f"{project_id}.{dataset_name}.{model_name}")

# Store the trained model object in the 'result' variable.
result = kmeans_model

In [ ]:
# prompt: #prompt: 1. Split df (using random state and test size 0.2) into test and training data for a K-means clustering algorithm store these as df_test and df_train. 2. Create a K-means cluster model using bigframes.ml.cluster KMeans with 5 clusters. 3. Save the model using the to_gbq method where the model name is project_id.dataset_name.model_name.
# df_train, df_test = train_test_split(bq_df, test_size=0.2, random_state = 42)
# kmeans = KMeans(n_clusters=5)
# kmeans.fit(df_train)
# kmeans.to_gbq(f"{project_id}.{dataset_name}.{model_name}")

# Split the DataFrame df into training and testing sets.
# We use train_test_split from bigframes.ml.model_selection, specifying the test_size and random_state for reproducibility.
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Create a KMeans clustering model.
# We instantiate KMeans from bigframes.ml.cluster, setting the number of clusters to 5.
kmeans_model = KMeans(n_clusters=5)

# Fit the KMeans model on the training data.
# The fit method trains the model using the features in df_train.
kmeans_model.fit(df_train)

# Save the trained model to BigQuery.
# The to_gbq method saves the model to the specified BigQuery path.
kmeans_model.to_gbq(f"{project_id}.{dataset_name}.{model_name}")

# Store the trained model object in the 'result' variable.
result = kmeans_model

In [ ]:
# prompt: 1. Call the K-means prediction model on the df dataframe, and store the results as predictions_df and show the first 10 records.

predictions_df = kmeans_model.predict(df)
result = predictions_df.head(10)

In [ ]:
# prompt: # prompt: 1. Call the K-means prediction model on the df dataframe, and store the results as predictions_df and show the first 10 records.
# predictions_df = kmeans.predict(df_test)
# predictions_df.head(10)

predictions_df = kmeans_model.predict(df)
result = predictions_df.head(10)

In [ ]:
# prompt: 1. Using predictions_df, and matplotlib, generate a scatterplot. 2. On the x-axis of the scatterplot, display days_since_last_order and on the y-axis, display average_spend from predictions_df. 3. Color by cluster. 4. The chart should be titled "Attribute grouped by K-means cluster."

import matplotlib.pyplot as plt

# Create the scatterplot
plt.figure(figsize=(10, 7))
scatter = plt.scatter(
    predictions_df['days_since_last_order'],
    predictions_df['average_spend'],
    c=predictions_df['CENTROID_ID'],  # Color by cluster
    cmap='viridis',  # Colormap for clusters
    alpha=0.6
)

# Add labels and title
plt.xlabel('Days Since Last Order')
plt.ylabel('Average Spend')
plt.title('Attribute grouped by K-means cluster')

# Add a colorbar for the clusters
plt.colorbar(scatter, label='Cluster ID')

# Show the plot
plt.show()

In [ ]:
# prompt: #prompt: 1. Using predictions_df, and matplotlib, generate a scatterplot. 2. On the x-axis of the scatterplot, display days_since_last_order and on the y-axis, display average_spend from predictions_df. 3. Color by cluster. 4. The chart should be titled "Attribute grouped by K-means cluster."
# import matplotlib.pyplot as plt
# # Create the scatter plot
# plt.figure(figsize=(10, 6))  # Adjust figure size as needed
# plt.scatter(predictions_df['days_since_last_order'], predictions_df['average_spend'], c=predictions_df['cluster'], cmap='viridis')
# # Customize the plot
# plt.title('Attribute grouped by K-means cluster')
# plt.xlabel('Days Since Last Order')
# plt.ylabel('Average Spend')
# plt.colorbar(label='Cluster ID')
# # Display the plot
# plt.show()

import matplotlib.pyplot as plt

# Create the scatterplot
plt.figure(figsize=(10, 7))
scatter = plt.scatter(
    predictions_df['days_since_last_order'],
    predictions_df['average_spend'],
    c=predictions_df['CENTROID_ID'],  # Color by cluster
    cmap='viridis',  # Colormap for clusters
    alpha=0.6
)

# Add labels and title
plt.xlabel('Days Since Last Order')
plt.ylabel('Average Spend')
plt.title('Attribute grouped by K-means cluster')

# Add a colorbar for the clusters
plt.colorbar(scatter, label='Cluster ID')

# Show the plot
plt.show()

In [ ]:
query = """
SELECT
 CONCAT('cluster ', CAST(centroid_id as STRING)) as centroid,
 average_spend,
 count_orders,
 days_since_last_order
FROM (
 SELECT centroid_id, feature, ROUND(numerical_value, 2) as value
 FROM ML.CENTROIDS(MODEL `{0}.{1}`)
)
PIVOT (
 SUM(value)
 FOR feature IN ('average_spend',  'count_orders', 'days_since_last_order')
)
ORDER BY centroid_id
""".format(dataset_name, model_name)

df_centroid = client.query(query).to_dataframe()
df_centroid.head()

In [ ]:
df_query = client.query(query).to_dataframe()
df_query.to_string(header=False, index=False)

cluster_info = []
for i, row in df_query.iterrows():
 cluster_info.append("{0}, average spend ${2}, count of orders per person {1}, days since last order {3}"
  .format(row["centroid"], row["count_orders"], row["average_spend"], row["days_since_last_order"]) )

cluster_info = (str.join("\n", cluster_info))
print(cluster_info)

In [ ]:
model = GenerativeModel("gemini-2.5-pro")

prompt = f"""
You're a creative brand strategist, given the following clusters, come up with \
creative brand persona, a catchy title, and next marketing action, \
explained step by step. Identify the cluster number, the title of the person, a persona for them and the next marketing step.

Clusters:
{cluster_info}

For each Cluster:
* Title:
* Persona:
* Next marketing step:
"""

responses = model.generate_content(
   prompt,
   generation_config={
      "temperature": 0.1,
      "max_output_tokens": 4000,
      "top_p": 1.0,
      "top_k": 40,
   }
)

print(responses.text)

In [ ]:
# prompt: **Cluster 1:**
# * **Title:** The Lapsed Loyalists
# * **Persona:** These customers have made a purchase in the past but haven't returned for an extended period. They likely had a positive experience but haven't been engaged recently.
# * **Next Marketing Step:**
#    1. **Re-engagement campaign:** Send personalized emails or targeted ads reminding them of their previous purchase and highlighting new products or promotions that might interest them.
#    2. **Offer exclusive discounts or incentives:** Motivate them to return with special offers or loyalty rewards.
#    3. **Personalized product recommendations:** Leverage purchase history and browsing behavior to suggest relevant products they might be interested in.
# **Cluster 2:**
# * **Title:** The Occasional Treaters
# * **Persona:** These customers make infrequent purchases but spend more when they do. They likely view the brand as a premium option for special occasions.
# * **Next Marketing Step:**
#    1. **Highlight exclusivity and premium value:** Emphasize the unique features and benefits of your products to justify the higher price point.
#    2. **Offer limited-time promotions or bundles:** Encourage larger purchases with special deals on high-value items or curated product sets.
#    3. **Create a sense of urgency and scarcity:** Promote limited-edition products or flash sales to encourage immediate action.
# **Cluster 3:**
# * **Title:** The One-and-Done Buyers
# * **Persona:** These customers have only made a single purchase and haven't returned. They might have had a neutral experience or haven't found a reason to come back.
# * **Next Marketing Step:**
#    1. **Gather feedback:** Send post-purchase surveys to understand their experience and identify areas for improvement.
#    2. **Offer personalized recommendations:** Based on their initial purchase, suggest complementary products or accessories to encourage further engagement.
#    3. **Showcase customer testimonials and social proof:** Highlight positive reviews and user-generat

import pandas as pd

# Create a DataFrame from the provided data
data = {
    'centroid': ['cluster 0', 'cluster 1', 'cluster 2', 'cluster 3', 'cluster 4'],
    'average_spend': [100.00, 50.00, 200.00, 75.00, 150.00],
    'count_orders': [5, 2, 10, 3, 7],
    'days_since_last_order': [30, 90, 10, 60, 20]
}
df_query = pd.DataFrame(data)

# Filter for Cluster 1
cluster_1_data = df_query[df_query['centroid'] == 'cluster 1']

# Extract the relevant values for Cluster 1
cluster_1_average_spend = cluster_1_data['average_spend'].iloc[0]
cluster_1_count_orders = cluster_1_data['count_orders'].iloc[0]
cluster_1_days_since_last_order = cluster_1_data['days_since_last_order'].iloc[0]

# Print the extracted values
print(f"Cluster 1 - Average Spend: ${cluster_1_average_spend:.2f}")
print(f"Cluster 1 - Count Orders: {cluster_1_count_orders}")
print(f"Cluster 1 - Days Since Last Order: {cluster_1_days_since_last_order}")